In [49]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
# read the file from folder and set 0 value values as nan
buildings = pd.read_csv('/Users/dannyqiu/Desktop/UIUC/IS445/week3/building_inventory.csv',
                                       na_values={'Year Acquired':0,
                                                  'Year Constructed':0,
                                                  'Square Footage':0})

In [51]:
# make a pivot table only contain rows as Congress dist, columns as Agnecy Name
# and value as totoal square footage
mydata=pd.pivot_table(buildings,
                                    index=['Congress Dist'],
                                    values=['Square Footage'],
                                    columns=['Agency Name'],
                                    aggfunc=np.sum)

In [52]:
mydata

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1219492.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

In [53]:
# This is the function to make connection with two graphs.
mySelectedLabel=ipywidgets.Label()
def on_selected(change):
    if len(change['owner'].selected)==1:
        i,j=change['owner'].selected[0]
        v = mydata.iloc[i,j] 
        mySelectedLabel.value = 'total square footage  = ' + str(v)
        mask=(buildings['Congress Dist'] == mydata.index[i])&(buildings['Agency Name']==mydata.columns.levels[1].to_list()[j])
        buildings_subset=buildings[mask]
        year_footage = buildings_subset.groupby('Year Acquired')['Square Footage'].sum()
        line.x=year_footage.index
        line.y=year_footage
    
        



In [54]:
#1 data--mydata--heatmap

#2 Scale--color scale
col_sc=bqplot.ColorScale(scheme="default")
x_sc=bqplot.OrdinalScale()
y_sc=bqplot.OrdinalScale()

#3 Axis
ax_col=bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right',label='SQ')
ax_x=bqplot.Axis(scale=x_sc,Label='Agency Name')
ax_y=bqplot.Axis(scale=y_sc,orientation='vertical',Label='District')

#4 Mark--heatmap
heat_map = bqplot.GridHeatMap(color = mydata,
                           row= mydata.index,
                           column=mydata.columns.levels[1].to_list(),
                           scales = {'color':col_sc, 'row':y_sc, 'column':x_sc},
                           interactions={'click':'select'},
                           anchor_style={'fill':'blue'},
                           selected_style={'opacity':1.0},
                           unselected_style={'opacity':0.8})

#fig and dashboard
heat_map.observe(on_selected, 'selected')
fig = bqplot.Figure(marks = [heat_map], axes=[ax_col, ax_x, ax_y],title='Total Square footage in each department by dist')
myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard

In [55]:
# Lines plot for the 'selected square footage by year'
year_footage = buildings.groupby('Year Acquired')['Square Footage'].sum()
#Scale
x_scl=bqplot.LinearScale()
y_scl=bqplot.LinearScale()

#3 Axis--x&y
ax_xl=bqplot.Axis(scale=x_scl,label='year')
ax_yl=bqplot.Axis(scale=y_scl,label='Square Footage',orientation='vertical')

#mark
line=bqplot.Lines(x=year_footage.index,y=year_footage,
                 scales={'x':x_scl,'y':y_scl})

figl=bqplot.Figure(marks=[line],axes=[ax_xl,ax_yl],title='Total Square footage by year')
line.observe(on_selected, 'selected')


In [56]:
# fig and dashboard with two components
fig.layout.min_width='500px'
figl.layout.min_width='500px'
figures=ipywidgets.HBox([fig,figl])

myDashboard=ipywidgets.VBox([mySelectedLabel,figures])
myDashboard